In [28]:
from sympy import *
from sympy import lambdify, DiracDelta
import numpy as np
import plotly.graph_objects as go
from IPython.display import display, Math
from scipy.integrate import odeint
from sympy.integrals.transforms import inverse_laplace_transform
from sympy import residue

Введите свою функцию $W_р$(s) разомкнутой системы:

In [49]:
# Define open-loop transfer function W_open(s)
s = Symbol('s')

k = Symbol('k', real = True)

T1 = Symbol('T1', real = True)
T2 = Symbol('T2', real = True)
T3 = Symbol('T3', real = True)

#↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
W_open = k * (0.25*s+1) / (s*(0.05*s+1)**2)
#↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑

# Display open-loop transfer function W_open(s)
display(Math('W_р(s) = ' + latex(W_open)))

# Print as LaTeX code for copy-paste to LaTeX document
print('Скопируйте код ниже и вставьте в свой LaTeX-файл:')
print('\\begin{equation}' + '\n' + '\t' + 'W_р(s) = ' +  latex(W_open) + '\n' + '\end{equation}')

<IPython.core.display.Math object>

Скопируйте код ниже и вставьте в свой LaTeX-файл:
\begin{equation}
	W_р(s) = \frac{k \left(0.25 s + 1\right)}{s \left(0.05 s + 1\right)^{2}}
\end{equation}


In [30]:
simple_fractions = Add.make_args(W_open.apart())

KeyboardInterrupt: 

\begin{array}{|c|c|}
\hline
  \text{Laplace transform} & \text{Z-transform} \\
\hline
   \\
   \dfrac{1}{s} & \dfrac{z}{z-1} \\
   \\
\hline
   \\
   \dfrac{1}{s^2} & \dfrac{T z}{(z-1)^2} \\
   \\
\hline
   \\
   \dfrac{1}{s+\alpha} & \dfrac{z}{z-e^{-\alpha T}} \\ 
   \\
\hline
   \\
   \dfrac{1}{(s+\alpha)^2} & \dfrac{T z e^{-\alpha T}}{(z-e^{-\alpha T})^2} \\
   \\
\hline
   \\
   \dfrac{\beta}{s^2+\beta^2} & \dfrac{z \sin(\beta T)}{z^2-2 z \cos(\beta T)+1} \\
   \\
\hline
   \\
   \dfrac{s}{s^2+\beta^2} & \dfrac{z^2 - z \cos(\beta T)}{z^2-2 z \cos(\beta T)+1} \\
   \\
\hline
   \\
   \dfrac{\beta}{(s + \alpha)^2 + \beta^2} & \dfrac{z e^{-\alpha T} \sin(\beta T)}{z^2-2 z e^{-\alpha T} \cos(\beta T)+e^{-2 \alpha T}} \\
   \\
\hline
   \\
   \dfrac{s + \alpha}{(s + \alpha)^2 + \beta^2} & \dfrac{z(z-e^{-\alpha T}\cos{\beta T})}{z^2-2 z e^{-\alpha T} \cos(\beta T)+e^{-2 \alpha T}} \\
   \\
\hline
\end{array}

In [ ]:
# Table of Laplace transform - Z-transform
z_transforms = []
z = Symbol('z')
T = Symbol('T', real = True)
for i in range(len(simple_fractions)):
    numerator = poly(simple_fractions[i].as_numer_denom()[0], s)
    denominator = poly(simple_fractions[i].as_numer_denom()[1], s)
    a = denominator.all_coeffs()[0]
    denominator = poly((denominator / a).simplify(), s)
    numerator = poly((numerator / a).simplify(), s)
    print('numerator = ', numerator)
    print('denominator = ', denominator)
    if numerator.degree(s) == 0 and denominator.degree(s) == 1:
        alpha = Symbol('alpha')
        alpha = solve(s + alpha - denominator, alpha)
        print(alpha)
        z_transforms.append(numerator * z / (z - exp(-alpha[0]*T)))
    elif numerator.degree(s) == 1 and denominator.degree(s) == 2:
        alpha = Symbol('alpha')
        beta = Symbol('beta')
        k1 = Symbol('k1')
        k2 = Symbol('k2')
        [alpha, beta, k1, k2] = solve((k1*(s + alpha) + k2*beta)/((s+alpha)**2 + beta**2) - simple_fractions[i], [alpha, beta, k1, k2])[0]
        z_transforms.append(k1 * z * (z - exp(-alpha*T)*cos(beta*T))/(z**2 - 2*exp(-alpha*T)*z*cos(beta*T) + exp(-2*alpha*T)))
        z_transforms.append(k2 * z * exp(-alpha*T)*sin(beta*T)/(z**2 - 2*exp(-alpha*T)*z*cos(beta*T) + exp(-2*alpha*T)))

# Make form as (s+\alpha)^2 + \beta^2 for simple_fraction
# simple_fractions[1].as_numer_denom()[0].factor() / simple_fractions[1].as_numer_denom()[1].factor()
z_transforms 

numerator =  Poly(-0.588235294117647, s, domain='RR')
denominator =  Poly(1.0*s + 4.0, s, domain='RR')
[4.00000000000000]
numerator =  Poly(0.588235294117647*s + 28.6764705882353, s, domain='RR')
denominator =  Poly(1.0*s**2 + 1.75*s + 25.0, s, domain='RR')


[-0.588235294117647*z/(z - exp(-4.0*T)),
 0.588235294117647*z*(z - exp(-0.875*T)*cos(4.9228421668788*T))/(z**2 - 2*z*exp(-0.875*T)*cos(4.9228421668788*T) + exp(-1.75*T)),
 5.72063124334079*z*exp(-0.875*T)*sin(4.9228421668788*T)/(z**2 - 2*z*exp(-0.875*T)*cos(4.9228421668788*T) + exp(-1.75*T))]

Билинейное преобразование

In [50]:
z = Symbol('z')
T = Symbol('T', real = True)
W_z = T*(z+1)/z/2*W_open.subs(s, 2/T * (z-1)/(z+1)).cancel()
numerator = nsimplify(W_z.as_numer_denom()[0]).collect(z)
denominator = nsimplify(W_z.as_numer_denom()[1]).collect(z)
W_z = (numerator / denominator).simplify().collect(z)
W_z

-25*T**3*k*(z + 1)*(2*T + z**3*(2*T + 1) + z**2*(6*T + 1) + z*(6*T - 1) - 1)/(2*z*(100*T**2 - 20*T + z**3*(-100*T**2 - 20*T - 1) + z**2*(-100*T**2 + 20*T + 3) + z*(100*T**2 + 20*T - 3) + 1))

In [85]:
w = Symbol('w')
W_w = W_z.subs(z, exp(I*w))
W_w_lambdify = lambdify([w, k, T], W_w, 'numpy')

In [89]:
# Lambdify function for H(w)

# Plot H(w)
w = np.linspace(0, np.pi, 10000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=w, y=np.log10(np.absolute(W_w_lambdify(w, 1, 0.01))), mode='lines', name='H(w)')) 
fig.update_layout(title='H(w)', xaxis_title='w', yaxis_title='H(w)')
fig.show()

Шаг дискретизации

In [ ]:
# T - sampling step
T = 0.1

In [ ]:
# Z-transform of the open-loop transfer function W_open(s)
# Z-transform is sum of residues in the poles of the transfer function of W_open(p)*z/(z-e^(p*T))

# Define z variable
z = Symbol('z') 

# Find poles of the transfer function of W_open(s)
poles = solve(W_open.as_numer_denom()[1], s)

# Find residues of the transfer function of W_open(s)
residue_s = residue(W_open*z/(z-exp(s*T)), s, 100000000)

residue_s

KeyboardInterrupt: 

In [ ]:
x = Symbol('x')
residue(2/sin(x), x, 0)

2